# Image Classification using Support Vector Machines (SVM)

Welcome! My name is Yasin Pourraisi.  
In this project, I will build a machine learning model to classify images of cats and dogs using the Support Vector Machines (SVM) algorithm. The goal is to explore how SVM and H.O.G. can be applied to computer vision tasks and evaluate its performance on distinguishing between cat and dog images.This model can also be trained to classify other types of images.

**H.O.G. (Histogram of Oriented Gradients):**
H.O.G. is a feature descriptor used in computer vision and image processing. It works by dividing an image into small regions (cells), calculating the gradient direction and magnitude for each pixel, and then creating a histogram of gradient directions for each cell. These histograms are combined to form a feature vector that describes the shape and appearance of objects in the image.

**SVM (Support Vector Machine):**
SVM is a supervised machine learning algorithm used for classification and regression tasks. It works by finding the optimal hyperplane that separates data points of different classes with the maximum margin. In image classification, SVM can be trained on feature vectors (like those from H.O.G.) to distinguish between categories.

**Connect with me:**  
- GitHub: [yasinpurraisi](https://github.com/yasinpurraisi)  
- Email: yasinpurraisi@gmail.com  
- Telegram: @yasinprsy


### Import Required Libraries

Let's start by importing all the necessary libraries for data processing, visualization, and building our model.

In [218]:
import numpy as np
import cv2
import json
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from joblib import dump

### Load Images and Labels

In this section, I load the image data and their corresponding labels from the dataset. The image file names and labels are stored in _annotation.json file, which you can [download here](https://cv-studio-accessible.s3.us-south.cloud-object-storage.appdomain.cloud/cats_dogs_images_.zip). For each image, I convert it to grayscale and resize it to 64 by 64 pixels to simplifies the algorithm and reduces computational requirements.
and i append 0 for cat and 1 for dog to labels.

In [219]:
train_images =[]
train_labels = []
images_dir = "cats_dogs_images/" 
with open("cats_dogs_images/_annotations.json","r") as d:
    data = json.load(d)
    class_object = data['labels']
    for key,value in data['annotations'].items():
        image = cv2.imread(images_dir+key)
        image = np.array(image).astype('uint8')
        image = cv2.resize(image,(64,64))
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

        hog_features, hog_images = hog(image,visualize=True,block_norm='L2-Hys',pixels_per_cell=(16, 16))
        
        label = 0 if value[0]['label'] == "cat" else 1
        
        train_images.append(hog_features)
        train_labels.append(label)


Create an array of the images and use the <code>np.vstack</code> to vertically stack arrays for wrangling,
and i convert <code>train_labels</code> to a numpy array with <code>integers</code> and reshape <code>train_labels</code> to a 2D column vector

finally i Concatenate the images and labels

In [220]:
train_array = np.array(train_images)
train_array = np.vstack(train_array)
labels_array = np.array(train_labels)
labels_array = labels_array.astype(int)
labels_array = labels_array.reshape((labels_array.size,1))
#Concatenate
train_df = np.concatenate([train_array, labels_array], axis = 1)

### Split Data

To evaluate the performance of my model, i split the dataset into two parts: a training set and a test set.

In [221]:
x = train_df[:, :-1]
y = train_df[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, shuffle=True)


### Hyperparameters in SVM

Hyperparameters are configuration settings used to control the behavior of a machine learning algorithm. In Support Vector Machines (SVM), the most important hyperparameters are:

<ul>
    <li><code>Kernel</code> Type:
        <ul>
            <li><code>Linear</code>: Used when data is linearly separable.</li>
            <li><code>RBF</code> (Radial Basis Function): Good for non-linear problems; most widely used.</li>
            <li><code>Poly</code> (Polynomial): Useful for data with polynomial relationships.</li>
            <li><code>Sigmoid</code>: Similar to neural networks’ activation function.</li>
        </ul>
    </li>
    <li><code>C</code> (Regularization Parameter):
        <ul>
            Controls the trade-off between achieving a low error on the training data and maintaining a large margin for separation.
            <li>High <code>C</code>: The model tries to classify all training examples correctly, possibly at the cost of a smaller margin (risk of overfitting).</li>
            <li>Low <code>C</code>: The model allows more misclassifications but aims for a larger margin (better generalization, less overfitting).widely used.</li>
        </ul>
    </li>
        <li><code>Gamma</code> for RBF Kernel:
        <ul>
            the spread of the kernel and, therefore, the decision region.
            <li>High <code>Gamma</code>: The model only considers points very close to each other as similar (narrow decision region, risk of overfitting).</li>
            <li>Low <code>Gamma</code>: The model considers points far away from each other as similar (wider decision region)</li>
        </ul>
    </li>
</ul>

i  select <code>C</code> : <code>[1, 10, 20, 30, 40]</code> and the best <code>kernel</code> : (Linear, RBF) by using the validation data.






In [222]:
params = {'kernel': ('linear', 'rbf'),'C': [1,10,20,30,40,100],'gamma': ['scale', 'auto', 0.01, 0.1, 1]}

default_estimator = SVC()

### Train the Model
i will train the model and try different kernels and parameter values using the function <code>GridSearchCV</code>. The resulting output will be the model that performs best on the validation data. 



In [223]:
svm = GridSearchCV(default_estimator, params, cv=5) # cv=5 : the training data is split into 5 parts
svm.fit(x_train,y_train)
best_parameters = svm.best_params_
print(best_parameters)

#predict labels for the test data
y_pred = svm.predict(x_test)

print("Accuracy: "+str(accuracy_score(y_test, y_pred)))

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Accuracy: 0.62


now i save the model

In [224]:
dump(svm.best_estimator_, 'svm_model.joblib')

['svm_model.joblib']

### Conclusion

In this project, I built an image classification model using Support Vector Machines (SVM) and Histogram of Oriented Gradients (H.O.G.) features to distinguish between cats and dogs. The workflow included loading and preprocessing images, extracting H.O.G. features, tuning SVM hyperparameters with GridSearchCV, and evaluating model performance. After training, the best model was saved for future use. While SVM with H.O.G. provides a solid baseline for image classification, further improvements can be achieved by optimizing feature extraction, scaling features, and experimenting with more advanced algorithms. This approach demonstrates the effectiveness of classical machine learning techniques in computer vision